## Self-Driving Car Engineer Nanodegree
---
### Deep Learning
#### Project: Build a Traffic Sign Recognition Classifier

In [ ]:
import requests, io, os
import random
import pickle, zipfile
import collections
import functools

import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow.contrib.layers import flatten

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.transform import resize
import matplotlib.image as mpimg

import matplotlib.pyplot as plt
%matplotlib inline

---
## Step 0: Load The Data

In [ ]:
# Prepare env and load files 

data_dir = 'data'
save_dir = 'model'
zip_file = os.path.join(data_dir, 'traffic-signs-data.zip')
zip_url = 'https://s3.amazonaws.com/video.udacity-data.com/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip'
if not os.path.isfile(zip_file):   
    print("Downloading {}".format(zip_url))
    if not os.path.exists('data'): os.mkdir(data_dir)
    r = requests.get(zip_url, allow_redirects=True)
    open(zip_file, 'wb').write(r.content)

if not os.path.isdir(save_dir): os.mkdir(save_dir)

In [ ]:
# Load pickled data

z = zipfile.ZipFile(zip_file)

train_p = pickle.load(z.open('train.p'))
test_p = pickle.load(z.open('test.p'))
    
X_train = train_p['features']
y_train = train_p['labels']
X_test  = test_p['features']
y_test  = test_p['labels']

# Load csv file with sign names
sign_names = pd.read_csv('signnames.csv')


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**


### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
# Number of training examples
n_train = len(X_train)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape 

# How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Exploratory visualization of the dataset

Sample sign names from dictionary

In [ ]:
sign_names.head()


Show 1 picture from every class

In [ ]:

plt.rcParams['figure.figsize'] = (12.0, 18.0)

p_cols = 5
p_rows = (n_classes / p_cols) + 1

# Plot one image for each label
for i in range(0, n_classes):
    sign_name = sign_names.loc[i].SignName
    idx_class = np.where(y_train == i)[0]
    rand_i = np.random.choice(idx_class)
    plt.subplot(p_rows, p_cols, i + 1)
    plt.imshow(X_train[rand_i])
    plt.ylabel(sign_name, fontsize=200/(np.max([20, len(sign_name)])))   
plt.show()

In [ ]:
def hist(sign_set):
    plt.figure(figsize=(25,4))
    plt.hist(sign_set, bins=n_classes)
    plt.title("Number of samples per sign type", loc='center')  
    plt.xlabel('Sign')
    plt.ylabel('Count')
    plt.plot()
    

In [ ]:
hist(y_train);

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Normalization

In [ ]:
# pixel values [0,1]. (images still in color)
def normalize(img):
    img_array = np.asarray(img)
    normalized = (img_array - img_array.min()) / (img_array.max() - img_array.min())
    return normalized

X_train = normalize(X_train)
X_test = normalize(X_test)


Split samples to validation and training sets

In [ ]:
X_train_split, X_validate_split, y_train_split, y_validate_split = train_test_split(X_train, y_train, 
                                                    test_size=0.2, 
                                                    random_state=111, 
                                                    stratify=y_train)



In [ ]:
# training split
hist(y_train_split)

In [ ]:
# validation split
hist(y_validate_split)

### Model Architecture

In [ ]:
Line = collections.namedtuple('Line', ['w', 'b', 'x'])

X_validate_split, y_validate_split = shuffle(X_validate_split, y_validate_split)

model_save_path = os.path.join(save_dir, 'traffic_classifier')
model_meta_path = os.path.join(save_dir, 'traffic_classifier.meta')

EPOCHS = 1 # 150
BATCH_SIZE = 128
LEARNING_RATE = 0.001

CONV_STRIDES = (1, 1, 1, 1)
CONV_PADDING = 'VALID'
POOL_KSIZE   = (1, 2, 2, 1)
POOL_STRIDES = (1, 2, 2, 1)
POOL_PADDING = 'VALID'

# Hyperparameters
MEAN = 0
STDDEV = 0.1

x = tf.placeholder(tf.float32, (None,) + image_shape)
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)

#tf.add_to_collection('x', x)
#tf.add_to_collection('y', y)
#tf.add_to_collection('keep_prob', keep_prob)

In [ ]:
# one hot encoding for all possible classes
one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
def conv2d(x, shape, mean, stddev):
    size = shape[-1]
    w = tf.Variable(tf.truncated_normal(shape=shape, mean = mean, stddev = stddev))
    b = tf.Variable(tf.zeros(size))
    conv = tf.nn.conv2d(x, w, strides=CONV_STRIDES, padding=CONV_PADDING) + b
    conv = tf.nn.relu(conv)
    conv = tf.nn.max_pool(conv, ksize=POOL_KSIZE, strides=POOL_STRIDES, padding=POOL_PADDING)
    return Line(w=w, b=b, x=conv)

In [ ]:
def dropout(x, shape, mean, stddev):
    size = shape[-1]
    w  = tf.Variable(tf.truncated_normal(shape=shape, mean = mean, stddev = stddev))
    b  = tf.Variable(tf.zeros(size))
    fc = tf.matmul(x, w) + b
    fc = tf.nn.relu(fc)
    fc = tf.nn.dropout(fc, keep_prob)
    return  Line(w=w, b=b, x=fc)

In [ ]:
def matmul(x, shape, mean, stddev):
    size = shape[-1]
    w  = tf.Variable(tf.truncated_normal(shape=shape, mean = mean, stddev = stddev))
    b  = tf.Variable(tf.zeros(size))
    res = tf.matmul(x, w) + b
    return  Line(w=w, b=b, x=res)

In [ ]:
def LeNet(x):    
    mean = MEAN
    stddev = STDDEV
    
    conv1 = conv2d(x, (5, 5, 3, 32), mean, stddev)
    conv2 = conv2d(conv1.x, (5, 5, 32, 64), mean, stddev)
    
    fc0 = flatten(conv2.x)
    fc1 = dropout(fc0, (1600, 1024), mean, stddev)
    fc2 = dropout(fc1.x, (1024,512), mean, stddev)
    
    logits = matmul(fc2.x, (512, 43), mean, stddev)
    weights = [conv1.w, conv2.w, fc1.w, fc2.w, logits.w]   
    
    tf.add_to_collection('logits', logits.x)
    
    return logits.x, weights

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
logits, weights = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)

# L2 Regularization 
regularizers = functools.reduce(lambda s,w : s + tf.nn.l2_loss(w), weights, 0.0)
L2_strength = 1e-6 # L2 values between 1E-2 and 1E-6 have been found to produce good results. (tutorial)
loss_operation = tf.reduce_mean(cross_entropy) + L2_strength * regularizers

optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
training_operation = optimizer.minimize(loss_operation)


In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.add_to_collection('accuracy_operation', accuracy_operation)

def make_batches(x, y):
    x_len = len(x)
    for offset in range(0, x_len, BATCH_SIZE):
        end = offset + BATCH_SIZE
        batch_x = x[offset:end] 
        batch_y = y[offset:end]
        yield batch_x, batch_y  

def training(x_train, y_train):
    sess = tf.get_default_session()
    for batch_x, batch_y in make_batches(x_train, y_train):
        sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
        
def evaluate(x_data, y_data):
    accuracy = 0
    loss = 0
    examples_size = len(x_data)
    sess = tf.get_default_session()
    for batch_x, batch_y in make_batches(x_data, y_data):
        batch_accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob:1.0})
        batch_loss = sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y, keep_prob:1.0})
        accuracy += (batch_accuracy * len(batch_x))
        loss += (batch_loss * len(batch_x))
    return accuracy / examples_size, loss / examples_size


In [ ]:
# TensorFlow to automatically choose an existing and supported device to run the operations 
# in case the specified one doesn't exist,
config = tf.ConfigProto(allow_soft_placement = True)
# Fraction of the overall amount of memory that each visible GPU should be allocated.
config.gpu_options.per_process_gpu_memory_fraction=0.4

In [ ]:
with tf.device('/gpu:0'):
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        
        print("Training...")
        for i in range(EPOCHS):
            X_train_split, y_train_split = shuffle(X_train_split, y_train_split)
            training(X_train_split, y_train_split)
            training_accuracy, training_loss = evaluate(X_train_split, y_train_split)
            validation_accuracy, validation_loss = evaluate(X_validate_split, y_validate_split)

            if i % 10 == 0:
                print(" EPOCH {} ...".format(i))
                print("  Training Accuracy = {:.5f}".format(training_accuracy))
                print("  Validation Accuracy = {:.5f}".format(validation_accuracy))
                print("  Training Loss = {:.5f}".format(training_loss))
                print("  Validation Loss = {:.5f}".format(validation_loss))

        saver = tf.train.Saver()
        path = os.path.join(save_dir, model_save_path)
        saver.save(sess, path)
        print("Model saved")

In [ ]:
# run model on testing samples
with tf.device('/cpu:0'):
    with tf.Session(config=config) as sess:
        loader = tf.train.import_meta_graph(model_meta_path)
        loader.restore(sess, tf.train.latest_checkpoint(save_dir))
        X_test = normalize(X_test)
        test_accuracy, test_loss = evaluate(X_test, y_test)
        print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images


### Load and Output the Images

In [ ]:
# Load new images sign names
misc_sign_names = pd.read_csv('miscsignnames.csv',index_col=0)
misc_sign_names.head()

In [ ]:
def load_image(filename): 
    img = mpimg.imread(filename)
    return resize(img, (32, 32), mode='constant', anti_aliasing=True)

In [ ]:
misc_dir = 'misc'

filenames = os.listdir(misc_dir)
misc_len = len(filenames)
misc_cols = 3
misc_rows = (misc_len / misc_cols) + 1

for index, filename in enumerate(filenames):
    sign_name = misc_sign_names.loc[filename].SignName
    img = load_image(os.path.join(misc_dir, filename))
    plt.subplot(misc_rows, misc_cols, index + 1)
    plt.imshow(img)
    plt.ylabel(sign_name, fontsize=12)   
plt.show()    
    


### Predict the Sign Type for Each Image

In [ ]:
def misc_image_graph(img, filename, classes, predict_confidence):
    
    fig = plt.figure(figsize=(12, 1))

    sub_img = fig.add_subplot(1, 2, 1)
    sub_img.imshow(img)
    sub_img.set_yticklabels([])
    sub_img.set_xticklabels([])

    bar_img = fig.add_subplot(1, 2, 2)
    width = 1      
    rect = bar_img.bar(classes, predict_confidence*100, width)
    bar_img.set_xlim(0, n_classes + 2)
    bar_img.set_ylim(0, 100)
    bar_img.set_ylabel('Confidence')
    bar_img.set_title('Scores')
    x_tick_marks = list(map(lambda c: 'id: {}'.format(classes[c]), range(0, len(classes))))
    bar_img.set_xticks(classes)
    x_tick_names = bar_img.set_xticklabels(x_tick_marks)
    plt.setp(x_tick_names, rotation=90, fontsize=8)
    plt.show()
    plt.close

filenames = os.listdir(misc_dir)

with tf.device('/cpu:0'):
    with tf.Session(config=config) as sess:
        loader = tf.train.import_meta_graph(model_meta_path)
        loader.restore(sess, tf.train.latest_checkpoint(save_dir))
        logits = tf.get_collection('logits')[0]

        print()
        top_k = 5
        for filename in filenames:
            img = load_image(os.path.join(misc_dir, filename))
            norm_img = normalize(img)
            test_prediction = tf.nn.softmax(logits)
            classification = sess.run(test_prediction, feed_dict = {x: [norm_img], keep_prob: 1.0})
            test_class = sess.run(tf.argmax(classification, 1))
            value, indices = sess.run(tf.nn.top_k(tf.constant(classification), k=top_k))

            predict_confidence = value.squeeze()
            indices = indices.squeeze()
            sign_name = misc_sign_names.loc[filename].SignName
            print('Sign Name: {} ({})'.format(sign_name, filename))
            
            for cl_id, confid in zip(indices, predict_confidence):
                cl_name = sign_names.loc[cl_id].SignName
                print(' Class_id:{0} ({1}), confidence:{2:.0%}'.format(cl_id, cl_name, confid))
                
            misc_image_graph(img, filename, indices, predict_confidence)
            print()    
                


### Analyze Performance

### Calculate the accuracy for these new images. 
#### Image 1
 filename: rs_01.jpg (Road work) was identify with confidence
#### Image 2
 filename: rs_02.jpg (Speed limit (70km/h)) was identify with confidence
#### Image 3
 filename: rs_03.jpg (Turn right ahead was identify with confidence
#### Image 4
 filename: rs_04.jpeg (Yield) was identify with confidence
#### Image 5
 filename: rs_05.jpg (Pedestrians Only) was identify with confidence
#### Image 6
 filename: rs_06.jpg (Right-of-way at the next intersection) was identify with confidence
#### Image 7
 filename: rs_07.jpg (Wild animals crossing) was identify with confidence
#### Image 8
 filename: rs_08.jpg (Priority road) was identify with confidence
#### Image 9
 filename: rs_09.jpg (Man with boat crossing) was identify with confidence
#### Image 10
 filename: rs_10.jpeg (Drunk man crossing) was identify with confidence
#### Image 11
 filename: rs_11.jpg (Wild animals crossing) was identify with confidence